In [27]:
%load_ext autoreload
%autoreload 2

import json
import io
import random
import boto3

import xml.etree.ElementTree as ET

from annotation_helpers import get_annotations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
bucket = 'sagemaker-facemasks-object-detection'
prefix = 'annotations/'

s3 = boto3.client('s3')
paginator = s3.get_paginator('list_objects')

In [29]:
annotations = []
for page in paginator.paginate(Bucket=bucket, Prefix='annotations/'):
    for c in page['Contents']:
        key = c['Key']
        if '.xml' in key:
            obj = s3.get_object(Bucket=bucket, Key=key)
            xml = ET.fromstring(obj['Body'].read())
            annot = get_annotations(xml)
            annot['source-ref'] = f's3://{bucket}/images/{annot["source-ref"]}'
            annotations.append(annot)

In [31]:
from PIL import Image
from io import BytesIO

def load_image(bucket, key):
    file_byte_string = s3.get_object(Bucket=bucket, Key=key)['Body'].read()
    return Image.open(BytesIO(file_byte_string))

In [42]:
bucket = 'sagemaker-facemasks-object-detection'
key = 'images/maksssksksss0.png'

img = load_image(bucket, key)

faces = []
for annot in annotations[0]['bounding-box']['annotations']:
    left = annot['left']
    top = annot['top']
    right = left + annot['width']
    bottom = top + annot['height']
    cls = annot['class_id']
    faces.append(img.crop((left, top, right, bottom)))